<a href="https://colab.research.google.com/github/mint-rin/HUFSdataAnalysis/blob/main/final_project02_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 한글 폰트를 설치

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-san

# 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

# 네이버 영화 리뷰 데이터 - 다운로드

In [ ]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
df = pd.read_csv('zootopia.csv')

# 데이터프레임의 크기
print(df.shape)

# 첫 5행을 출력
df.head()

(10, 3)


,title,user,review
0,주토피아 (Zootopia),trav****,"\n★★★★☆""사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작.""픽사는 디즈니..."
1,주토피아 시사회 후기 - '편견'과 '다르다' 대해서 (스포주의),dlax****,\n※글 작성 관련하여 스포가 조금 있으니 피하고자 하시는 분은 뒤로가기를...--...
2,[영화/시사회] 주토피아 _ 세상은 생각보다 단순하다,dbw8****,\n​​첫인상은 솔직히 별로였다. 행여나 끝나는 순간까지 '노력으로 환경을 이겨내는...
3,주토피아를 보고 드는 의문점 (중간에 스포),mira****,"\n우선, 영화는 영상과 설정과 케릭터만 매력적이고 스토리는 그닥... 이라는 생각..."
4,불평등을 인정하고 내 삶을 능동적으로 창조하자,dldl****,\n취미로 글쓰는 중 입니다~ 많이 미흡해도 이쁘게 봐주세요!!한 사람의 인생을 멀...


# 한글 형태소 분석


## 1) KoNLPy 설치 (코랩에서 실습할 경우)

In [ ]:
# 코랩에 KoNLPY 패키지를 설치
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 2) KoNLPy 한글 형태소 분석

In [ ]:
sample_text = str(df['review'])
print(type(sample_text))
print(sample_text)

<class 'str'>
0    \n★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니...
1    \n※글 작성 관련하여 스포가 조금 있으니 피하고자 하시는 분은 뒤로가기를...--...
2    \n​​첫인상은 솔직히 별로였다. 행여나 끝나는 순간까지 '노력으로 환경을 이겨내는...
3    \n우선, 영화는 영상과 설정과 케릭터만 매력적이고 스토리는 그닥... 이라는 생각...
4    \n취미로 글쓰는 중 입니다~ 많이 미흡해도 이쁘게 봐주세요!!한 사람의 인생을 멀...
5    \n\n당근 농장이 있는 촌에 살고 있던 토끼는세상을 좋게 만들겠다는 꿈을 갖고이상...
6    \n   지니퍼 굿윈의 영화 주토피아는, 정말 잘 만든 애니메이션이다. 지니퍼 굿윈...
7    \n\n\n\n\n주토피아\n\n감독\n바이론 하워드, 리치 무어\n출연\n지니퍼 ...
8    \n지니퍼 굿윈(주디 홉스), 제이슨 베이트먼(닉 와일드), 샤키라(가젤), 이드리...
9    \n영화 시사회에 다녀왔다.뭔가 검사외전의 애니메이션 버전일 것 같은정의의 편과 사...
Name: review, dtype: object


In [ ]:
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

In [ ]:
# (단어, 품사) 추출
tokens = okt.pos(sample_text)
pprint(tokens)

[('0', 'Number'),
 ('\\', 'Punctuation'),
 ('n', 'Alpha'),
 ('★★★★☆', 'Foreign'),
 ('"', 'Punctuation'),
 ('사회', 'Noun'),
 ('문제', 'Noun'),
 ('조차도', 'Josa'),
 ('따뜻하게', 'Adjective'),
 ('포', 'Noun'),
 ('용하는', 'Adjective'),
 ('디즈니', 'Noun'),
 ('의', 'Josa'),
 ('또다른', 'Modifier'),
 ('걸작', 'Noun'),
 ('."', 'Punctuation'),
 ('픽사', 'Noun'),
 ('는', 'Josa'),
 ('디즈니', 'Noun'),
 ('...', 'Punctuation'),
 ('\n', 'Foreign'),
 ('1', 'Number'),
 ('\\', 'Punctuation'),
 ('n', 'Alpha'),
 ('※', 'Foreign'),
 ('글', 'Noun'),
 ('작성', 'Noun'),
 ('관련', 'Noun'),
 ('하여', 'Verb'),
 ('스포', 'Noun'),
 ('가', 'Josa'),
 ('조금', 'Noun'),
 ('있으니', 'Adjective'),
 ('피하', 'Noun'),
 ('고자', 'Noun'),
 ('하시는', 'Verb'),
 ('분', 'Noun'),
 ('은', 'Josa'),
 ('뒤', 'Noun'),
 ('로', 'Josa'),
 ('가기', 'Noun'),
 ('를', 'Josa'),
 ('...--...', 'Punctuation'),
 ('\n', 'Foreign'),
 ('2', 'Number'),
 ('\\', 'Punctuation'),
 ('n', 'Alpha'),
 ('\u200b\u200b', 'Foreign'),
 ('첫인상', 'Noun'),
 ('은', 'Josa'),
 ('솔직히', 'Adjective'),
 ('별로', 'Noun'),
 ('였다',

In [ ]:
# 단어 ONLY 추출
tokens = okt.morphs(sample_text)
pprint(tokens)

['0',
 '\\',
 'n',
 '★★★★☆',
 '"',
 '사회',
 '문제',
 '조차도',
 '따뜻하게',
 '포',
 '용하는',
 '디즈니',
 '의',
 '또다른',
 '걸작',
 '."',
 '픽사',
 '는',
 '디즈니',
 '...',
 '\n',
 '1',
 '\\',
 'n',
 '※',
 '글',
 '작성',
 '관련',
 '하여',
 '스포',
 '가',
 '조금',
 '있으니',
 '피하',
 '고자',
 '하시는',
 '분',
 '은',
 '뒤',
 '로',
 '가기',
 '를',
 '...--...',
 '\n',
 '2',
 '\\',
 'n',
 '\u200b\u200b',
 '첫인상',
 '은',
 '솔직히',
 '별로',
 '였다',
 '.',
 '행여',
 '나',
 '\xa0',
 '끝나는',
 '순간',
 '까지',
 "'",
 '노력',
 '으로',
 '환경',
 '을',
 '이겨내는',
 '...',
 '\n',
 '3',
 '\\',
 'n',
 '우선',
 ',',
 '영화',
 '는',
 '영상',
 '과',
 '설정',
 '과',
 '케릭',
 '터',
 '만',
 '매력',
 '적',
 '이고',
 '스토리',
 '는',
 '그닥',
 '...',
 '이라는',
 '생각',
 '...',
 '\n',
 '4',
 '\\',
 'n',
 '취미',
 '로',
 '글',
 '쓰는',
 '중',
 '입니다',
 '~',
 '많이',
 '미흡해도',
 '이쁘게',
 '봐주세요',
 '!!',
 '한',
 '사람',
 '의',
 '인생',
 '을',
 '멀',
 '...',
 '\n',
 '5',
 '\\',
 'n',
 '\\',
 'n',
 '당근',
 '농장',
 '이',
 '있는',
 '촌',
 '에',
 '살',
 '고',
 '있던',
 '토끼는',
 '세상',
 '을',
 '좋게',
 '만들겠다는',
 '꿈',
 '을',
 '갖고',
 '이상',
 '...',
 '\n',
 '6',
 '\\',
 

In [ ]:
# 명사 ONLY 추출
tokens = okt.nouns(sample_text)
pprint(tokens)

['사회',
 '문제',
 '포',
 '디즈니',
 '걸작',
 '픽사',
 '디즈니',
 '글',
 '작성',
 '관련',
 '스포',
 '조금',
 '피하',
 '고자',
 '분',
 '뒤',
 '가기',
 '첫인상',
 '별로',
 '행여',
 '순간',
 '노력',
 '환경',
 '우선',
 '영화',
 '영상',
 '설정',
 '케릭',
 '터',
 '매력',
 '스토리',
 '그닥',
 '생각',
 '취미',
 '글',
 '중',
 '사람',
 '인생',
 '당근',
 '농장',
 '촌',
 '살',
 '세상',
 '꿈',
 '이상',
 '굿윈',
 '영화',
 '토피아',
 '정말',
 '애니메이션',
 '굿윈',
 '토피아',
 '감독',
 '바이론',
 '하워드',
 '리치',
 '무어',
 '출연',
 '굿윈',
 '홉스',
 '제이슨',
 '베이트',
 '닉',
 '와일드',
 '샤키라',
 '가젤',
 '이드',
 '리',
 '영화',
 '시사회',
 '뭔가',
 '검사',
 '외전',
 '애니메이션',
 '버전',
 '일',
 '것',
 '정의',
 '편',
 '사']


## 3) 텍스트 전처리

In [ ]:
# 누락 데이터를 제거
review_data = df['review'].dropna().values

# 학습 속도를 고려하여, 1000개의 샘플을 선택하여 추출
review_data = review_data[:1000]

# 첫 번째 데이터
print(review_data[0])


★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마법의 숲]같은 프린세스물을 만들었기에 하는 소리도 아니고, 디즈니가 [주먹왕 랄프]같이 전성기의 픽사처럼 통통튀는 상상력을 토대로한 작품을 만들었기 때문에 하는 소리도 아니다. 최근 픽사는 디즈니가 줄곧 그래왔던것처럼 '가족'이나 '감정' 같은 다소 개인적인 주제로 작품에 접근하고 있는 반면에 ([인사이드 아웃]이나 [굿 다이노]), 디즈니는 전성기 시절 픽사가 그랬던것 처럼 '정치적'인 주제로 작품에 접근하고 있다. '정치적'이라는 단어가 다소 생소하게 느껴질텐데, 디즈니가 펑크 록밴드처럼 '대통령 엿먹어!' '전쟁 반대!' 이런 구호를 외친다는것이 아니라 사회 시스템에 대해 관심을 가지기 시작했다는 의미이다. 이 뜻을 설명하기 위해서는 잠시 2000년대 중반으로 돌아가서 픽사의 전성기 시절을 되돌아 보아야 한다. 그때 당시 평론가들이 픽사 스튜디오에게 내린 주된 평은 이것이였다. '가장 정치적이면서 가장 정치적이지 않은 회사.' 여러분이 재밌게 보았던 [인크레더블]이나 [월E]같은 작품들을 떠올려보자. [인크레더블]은 '히어로 패밀리'라는 큰 주제를 가지고 있지만 은연중에 고객들을 등쳐먹고 있는 보험회사를 비판하기도 하고, [월E]에서는 대기업의 독점이 사회 시스템에 얼마나 악영향을 미치고 있는지를 보여주고 있다. 잘 생각해보면 픽사 작품에서 '숲'은 가족영화의 형태를 띄고 있지만 숲을 구성하고 있는 '나무'들은 사회를 바라보는 시니컬한 시선의 주제들로 구성되어있다. 픽사는 그런 영화를 만들던 회사였다. 그런데 픽사가 디즈니의 산하회사로 들어가게된 이후 픽사와 디즈니간의 교류가 잦아지면서 점차 둘을 구분하던 경계선이 흐릿해졌다. 이제는 픽사가 디즈니처럼 보이고, 디즈니가 픽사처럼 보이는 지경까지 이르게 된것이다. 하지만 놀랍게도 서

In [ ]:
# 두 글자 이상의 명사를 사용 (한 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 1:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

100%|██████████| 10/10 [00:02<00:00,  3.52it/s]

10
사회 문제 디즈니 걸작 픽사 디즈니 디즈니 픽사 간다 디즈니 픽사 관계 요약 아마 픽사 메리다 마법 프린세스 소리 디즈니 주먹 랄프 전성기 픽사 통통 상상력 토대 작품 때문 소리 최근 픽사 디즈니 줄곧 가족 이나 감정 다소 개인 주제 작품 접근 반면 인사 이드 아웃 이나 다이노 디즈니 전성기 시절 픽사 처럼 정치 주제 작품 접근 정치 단어 다소 디즈니 펑크 밴드 대통령 엿먹어 전쟁 반대 구호 사회 시스템 대해 관심 시작 의미 설명 잠시 중반 픽사 전성기 시절 보아 그때 당시 평론가 픽사 스튜디오 가장 정치 면서 가장 정치 회사 여러분 인크레더블 이나 작품 인크레더블 히어로 패밀리 주제 가지 은연중 고객 보험 회사 비판 대기업 독점 사회 시스템 얼마나 영향 생각 픽사 작품 가족영화 형태 구성 나무 사회 시선 주제 구성 픽사 영화 회사 픽사 디즈니 회사 이후 픽사 디즈니 교류 점차 구분 경계 흐릿 이제 픽사 디즈니 보이 디즈니 픽사 지경 서로 아이덴티티 결과 시너지 실로 사실 인사 이드 아웃 토피아 증명 토피아 우선 동안 디즈니 영화 장점 통통 작품 매력 캐릭터 일가견 디즈니 이번 역시 캐릭터 통해 성공 스토리 텔링 마치 최대한 사랑 범죄 스릴러 개인 사건 사실 흑막 전개 동안 토피아 그것 인상 우선 장르 특성 아이 타겟 애니메이션 시도 그동안 보지 무엇 전체 소재 배경 자체 때문 동물 의인화 애니메이션 동물 극도 진화 현재 인간 사회 동시 동물 진보 사회 본능 설정 소재 이야기 개성 또한 영화 요소 고려 캐릭터 행동 자체 앞서 디즈니 매력 캐릭터 창조 일가견 때문 캐릭터 세계 마치 때문 창조 토피아 세계 매력 토피아 디즈니 특유 장점 작품 아이러니 동시 이전 디즈니 영화 다소 다른 양상 주먹 랄프 픽사 스타일 겨울왕국 전형 디즈니 스타일 가지 토피아 모습 모두 스타일 가지 토피아 분위기 세계 이면 의식 자리 잡고 태초 동물 약자 사냥 포식자 시간 동물 진화 서로 다른 동물 토피아 포식자 약자 구분 모토 토대 발전 도시 불구 포식자 대한 약자 공포 조사 사건

## 4) TF-IDF 벡터로 표현

In [ ]:
# 사이킷런 패키지 활용
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(10, 985)
  (0, 668)	0.02222680493835707
  (0, 926)	0.017746694686413378
  (0, 651)	0.025405493748610525
  (0, 160)	0.025405493748610525
  (0, 333)	0.02988560400055422
  (0, 975)	0.02988560400055422
  (0, 476)	0.02988560400055422
  (0, 223)	0.025405493748610525
  (0, 200)	0.02988560400055422
  (0, 260)	0.02988560400055422
  (0, 127)	0.025405493748610525
  (0, 118)	0.02988560400055422
  (0, 51)	0.02988560400055422
  (0, 843)	0.02222680493835707
  (0, 283)	0.02988560400055422
  (0, 904)	0.02988560400055422
  (0, 745)	0.017746694686413378
  (0, 33)	0.02988560400055422
  (0, 297)	0.025405493748610525
  (0, 909)	0.025405493748610525
  (0, 905)	0.02988560400055422
  (0, 840)	0.025405493748610525
  (0, 384)	0.02988560400055422
  (0, 789)	0.02988560400055422
  (0, 77)	0.02988560400055422
  :	:
  (0, 652)	0.08965681200166266
  (0, 7)	0.025405493748610525
  (0, 784)	0.02988560400055422
  (0, 836)	0.02988560400055422
  (0, 236)	0.08873347343206689
  (0, 712)	0.15558763456849947
  (0, 883)	0.05977

In [ ]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

985
{'사회': 423, '문제': 309, '디즈니': 232, '걸작': 37, '픽사': 923}


In [ ]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() }

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{423: '사회', 309: '문제', 232: '디즈니', 37: '걸작', 923: '픽사'}


In [ ]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

'이유 하나 이기 누구 발휘 효과 수용 두운 대하 마음 기분 극장 경우 추천 메세지 평가 정말 거리 목적 평화 평등 추구 분열 증오 공포정치 결말 우리 주지 이익 행사 예전 이슈 지금 발자취 인류 역사 정당화 과연 질문 한가지 관객 폭력 차별 억압 소수 다수 상황 달리 옛날 조사 공포 대한 불구 도시 발전 모토 시간 포식자 사냥 약자 태초 잡고 자리 의식 이면 분위기 모두 모습 전형 겨울왕국 스타일 양상 다른 이전 아이러니 특유 세계 창조 앞서 행동 고려 요소 또한 개성 이야기 설정 본능 진보 동시 인간 현재 진화 극도 의인화 동물 자체 배경 소재 전체 무엇 보지 그동안 시도 애니메이션 타겟 아이 특성 장르 인상 그것 전개 흑막 사건 스릴러 범죄 사랑 최대한 마치 텔링 스토리 성공 통해 역시 이번 일가견 캐릭터 매력 장점 동안 우선 증명 토피아 사실 실로 시너지 결과 아이덴티티 서로 지경 보이 이제 흐릿 경계 구분 점차 교류 이후 영화 시선 나무 구성 형태 가족영화 생각 영향 얼마나 독점 대기업 비판 보험 고객 은연중 가지 패밀리 히어로 인크레더블 여러분 회사 면서 가장 스튜디오 평론가 당시 그때 보아 중반 잠시 설명 의미 시작 관심 대해 시스템 구호 반대 전쟁 엿먹어 대통령 밴드 펑크 단어 정치 처럼 시절 다이노 아웃 이드 인사 반면 접근 주제 개인 다소 감정 이나 가족 줄곧 최근 때문 작품 토대 상상력 통통 전성기 랄프 주먹 소리 프린세스 마법 메리다 아마 요약 관계 간다 픽사 걸작 디즈니 문제 사회'

# Topic Modeling


## 1) LDA (LatentDirichletAllocation)

In [ ]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)

# TF-IDF 벡터를 입력하여 모델 학습
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [ ]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 2157개의 단어)
print(lda.components_.shape)

(2, 985)


In [ ]:
# 2157개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('토피아', 1.37), ('영화', 1.259), ('홉스', 1.066), ('경찰', 1.018), ('사건', 0.946), ('동물', 0.932), ('캐릭터', 0.875), ('실종', 0.865), ('재미', 0.844), ('시사회', 0.838)]
토픽 유형 2: [('세상', 1.212), ('영화', 1.09), ('그녀', 1.033), ('생각', 1.015), ('동물', 1.014), ('픽사', 1.003), ('우리', 0.979), ('차별', 0.978), ('경찰', 0.978), ('토피아', 0.975)]


## 2) pyLDAvis 시각화

In [ ]:
# pyLDAvis 설치
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

# Sentimental Analysis (감성 분석)


## 1) 머신러닝 알고리즘을 활용하여, 긍정/부정 감성을 분류

In [ ]:
labels = df['review'].iloc[:1000].values
print(labels.shape)
print(labels[:5])

(10,)
['\n★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마법의 숲]같은 프린세스물을 만들었기에 하는 소리도 아니고, 디즈니가 [주먹왕 랄프]같이 전성기의 픽사처럼 통통튀는 상상력을 토대로한 작품을 만들었기 때문에 하는 소리도 아니다. 최근 픽사는 디즈니가 줄곧 그래왔던것처럼 \'가족\'이나 \'감정\' 같은 다소 개인적인 주제로 작품에 접근하고 있는 반면에 ([인사이드 아웃]이나 [굿 다이노]), 디즈니는 전성기 시절 픽사가 그랬던것 처럼 \'정치적\'인 주제로 작품에 접근하고 있다. \'정치적\'이라는 단어가 다소 생소하게 느껴질텐데, 디즈니가 펑크 록밴드처럼 \'대통령 엿먹어!\' \'전쟁 반대!\' 이런 구호를 외친다는것이 아니라 사회 시스템에 대해 관심을 가지기 시작했다는 의미이다. 이 뜻을 설명하기 위해서는 잠시 2000년대 중반으로 돌아가서 픽사의 전성기 시절을 되돌아 보아야 한다. 그때 당시 평론가들이 픽사 스튜디오에게 내린 주된 평은 이것이였다. \'가장 정치적이면서 가장 정치적이지 않은 회사.\' 여러분이 재밌게 보았던 [인크레더블]이나 [월E]같은 작품들을 떠올려보자. [인크레더블]은 \'히어로 패밀리\'라는 큰 주제를 가지고 있지만 은연중에 고객들을 등쳐먹고 있는 보험회사를 비판하기도 하고, [월E]에서는 대기업의 독점이 사회 시스템에 얼마나 악영향을 미치고 있는지를 보여주고 있다. 잘 생각해보면 픽사 작품에서 \'숲\'은 가족영화의 형태를 띄고 있지만 숲을 구성하고 있는 \'나무\'들은 사회를 바라보는 시니컬한 시선의 주제들로 구성되어있다. 픽사는 그런 영화를 만들던 회사였다.\xa0그런데 픽사가 디즈니의 산하회사로 들어가게된 이후 픽사와 디즈니간의 교류가 잦아지면서 점차 둘을 구분하던 경계선이 흐릿해졌다. 이제는 픽사가 디즈니처럼 보이고, 디즈니가 픽사

In [ ]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습
lr.fit(review_tfid, labels)

LogisticRegression()

In [ ]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

['\n★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마법의 숲]같은 프린세스물을 만들었기에 하는 소리도 아니고, 디즈니가 [주먹왕 랄프]같이 전성기의 픽사처럼 통통튀는 상상력을 토대로한 작품을 만들었기 때문에 하는 소리도 아니다. 최근 픽사는 디즈니가 줄곧 그래왔던것처럼 \'가족\'이나 \'감정\' 같은 다소 개인적인 주제로 작품에 접근하고 있는 반면에 ([인사이드 아웃]이나 [굿 다이노]), 디즈니는 전성기 시절 픽사가 그랬던것 처럼 \'정치적\'인 주제로 작품에 접근하고 있다. \'정치적\'이라는 단어가 다소 생소하게 느껴질텐데, 디즈니가 펑크 록밴드처럼 \'대통령 엿먹어!\' \'전쟁 반대!\' 이런 구호를 외친다는것이 아니라 사회 시스템에 대해 관심을 가지기 시작했다는 의미이다. 이 뜻을 설명하기 위해서는 잠시 2000년대 중반으로 돌아가서 픽사의 전성기 시절을 되돌아 보아야 한다. 그때 당시 평론가들이 픽사 스튜디오에게 내린 주된 평은 이것이였다. \'가장 정치적이면서 가장 정치적이지 않은 회사.\' 여러분이 재밌게 보았던 [인크레더블]이나 [월E]같은 작품들을 떠올려보자. [인크레더블]은 \'히어로 패밀리\'라는 큰 주제를 가지고 있지만 은연중에 고객들을 등쳐먹고 있는 보험회사를 비판하기도 하고, [월E]에서는 대기업의 독점이 사회 시스템에 얼마나 악영향을 미치고 있는지를 보여주고 있다. 잘 생각해보면 픽사 작품에서 \'숲\'은 가족영화의 형태를 띄고 있지만 숲을 구성하고 있는 \'나무\'들은 사회를 바라보는 시니컬한 시선의 주제들로 구성되어있다. 픽사는 그런 영화를 만들던 회사였다.\xa0그런데 픽사가 디즈니의 산하회사로 들어가게된 이후 픽사와 디즈니간의 교류가 잦아지면서 점차 둘을 구분하던 경계선이 흐릿해졌다. 이제는 픽사가 디즈니처럼 보이고, 디즈니가 픽사처럼 보이는

## 2) Part 1에서 수집한 리뷰를 긍정, 부정으로 감성 분류 예측

In [ ]:
# 첫 번째 리뷰를 선택
test_sample = df['review'][0]
test_sample

'\n★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마법의 숲]같은 프린세스물을 만들었기에 하는 소리도 아니고, 디즈니가 [주먹왕 랄프]같이 전성기의 픽사처럼 통통튀는 상상력을 토대로한 작품을 만들었기 때문에 하는 소리도 아니다. 최근 픽사는 디즈니가 줄곧 그래왔던것처럼 \'가족\'이나 \'감정\' 같은 다소 개인적인 주제로 작품에 접근하고 있는 반면에 ([인사이드 아웃]이나 [굿 다이노]), 디즈니는 전성기 시절 픽사가 그랬던것 처럼 \'정치적\'인 주제로 작품에 접근하고 있다. \'정치적\'이라는 단어가 다소 생소하게 느껴질텐데, 디즈니가 펑크 록밴드처럼 \'대통령 엿먹어!\' \'전쟁 반대!\' 이런 구호를 외친다는것이 아니라 사회 시스템에 대해 관심을 가지기 시작했다는 의미이다. 이 뜻을 설명하기 위해서는 잠시 2000년대 중반으로 돌아가서 픽사의 전성기 시절을 되돌아 보아야 한다. 그때 당시 평론가들이 픽사 스튜디오에게 내린 주된 평은 이것이였다. \'가장 정치적이면서 가장 정치적이지 않은 회사.\' 여러분이 재밌게 보았던 [인크레더블]이나 [월E]같은 작품들을 떠올려보자. [인크레더블]은 \'히어로 패밀리\'라는 큰 주제를 가지고 있지만 은연중에 고객들을 등쳐먹고 있는 보험회사를 비판하기도 하고, [월E]에서는 대기업의 독점이 사회 시스템에 얼마나 악영향을 미치고 있는지를 보여주고 있다. 잘 생각해보면 픽사 작품에서 \'숲\'은 가족영화의 형태를 띄고 있지만 숲을 구성하고 있는 \'나무\'들은 사회를 바라보는 시니컬한 시선의 주제들로 구성되어있다. 픽사는 그런 영화를 만들던 회사였다.\xa0그런데 픽사가 디즈니의 산하회사로 들어가게된 이후 픽사와 디즈니간의 교류가 잦아지면서 점차 둘을 구분하던 경계선이 흐릿해졌다. 이제는 픽사가 디즈니처럼 보이고, 디즈니가 픽사처럼 보이는 

In [ ]:
# 한글을 제외하고 전부 제거
test_sample = re.sub(r"[^가-힣]", "", test_sample)
test_sample

'사회문제조차도따뜻하게포용하는디즈니의또다른걸작픽사는디즈니를닮아가고디즈니는픽사를닮아간다년대의디즈니와픽사의관계를한줄로요약하자면아마이럴것이다이것은단순히픽사가메리다와마법의숲같은프린세스물을만들었기에하는소리도아니고디즈니가주먹왕랄프같이전성기의픽사처럼통통튀는상상력을토대로한작품을만들었기때문에하는소리도아니다최근픽사는디즈니가줄곧그래왔던것처럼가족이나감정같은다소개인적인주제로작품에접근하고있는반면에인사이드아웃이나굿다이노디즈니는전성기시절픽사가그랬던것처럼정치적인주제로작품에접근하고있다정치적이라는단어가다소생소하게느껴질텐데디즈니가펑크록밴드처럼대통령엿먹어전쟁반대이런구호를외친다는것이아니라사회시스템에대해관심을가지기시작했다는의미이다이뜻을설명하기위해서는잠시년대중반으로돌아가서픽사의전성기시절을되돌아보아야한다그때당시평론가들이픽사스튜디오에게내린주된평은이것이였다가장정치적이면서가장정치적이지않은회사여러분이재밌게보았던인크레더블이나월같은작품들을떠올려보자인크레더블은히어로패밀리라는큰주제를가지고있지만은연중에고객들을등쳐먹고있는보험회사를비판하기도하고월에서는대기업의독점이사회시스템에얼마나악영향을미치고있는지를보여주고있다잘생각해보면픽사작품에서숲은가족영화의형태를띄고있지만숲을구성하고있는나무들은사회를바라보는시니컬한시선의주제들로구성되어있다픽사는그런영화를만들던회사였다그런데픽사가디즈니의산하회사로들어가게된이후픽사와디즈니간의교류가잦아지면서점차둘을구분하던경계선이흐릿해졌다이제는픽사가디즈니처럼보이고디즈니가픽사처럼보이는지경까지이르게된것이다하지만놀랍게도서로의아이덴티티를뒤바꾼결과그시너지는실로엄청났다그사실은인사이드아웃과주토피아가증명하고있다주토피아는우선그동안디즈니영화에서느꼈던독특한장점들이통통튀는작품이다매력적인캐릭터를만드는데일가견이된디즈니는이번에도역시사랑스러운캐릭터를통해성공적인스토리텔링을이끌어내고있다마치최대한사랑스럽게풀어낸범죄스릴러를보는것같다개인의사건이사실은큰흑막에가려져있었다는전개는그동안많이봐왔었던것이지만주토피아에서는그것이식상해보인다던가틀에박혀있다는인상을받지않았다우선장르특성상어린아이들을타겟층으로한애니메이션에서이러한시도는그동안보지못했던것이였고무엇보다전체적인소재와배경자

In [ ]:
# 두 글자 이상의 명사만을 추출

tokens = okt.nouns(test_sample)
cleaned_tokens = []
for word in tokens:
    if len(word) > 1:
        cleaned_tokens.append(word)
    else:
        pass
cleaned_review = " ".join(cleaned_tokens)

cleaned_review

'사회 문제 디즈니 걸작 픽사 디즈니 디즈니 픽사 간다 년대 디즈니 픽사 관계 요약 픽사 메리다 마법 프린세스 소리 도아 디즈니 주먹 랄프 전성기 픽사 통통 상상력 토대 작품 때문 소리 도아 최근 픽사 디즈니 줄곧 가족 감정 개인 주제 작품 접근 반면 인사이드아웃 다이노 디즈니 전성기 시절 픽사 정치 주제 작품 접근 정치 단어 다소 디즈니 펑크록 밴드 대통령 엿먹어 쟁반 구호 사회 시스템 대해 관심 가지 시작 의미 설명 위해 잠시 년대 중반 픽사 전성기 시절 그때 당시 평론가 픽사 스튜디오 정치 이면 가장 정치 회사 여러분 인크레더블 작품 인크레더블 히어로 패밀리 주제 가지 연중 고객 보험 회사 비판 기도 대기업 독점 이사회 시스템 얼마나 영향 을미 생각 픽사 작품 가족영화 형태 구성 나무 사회 시선 주제 구성 픽사 영화 회사 픽사 디즈니 회사 이후 픽사 디즈니 교류 점차 구분 경계 흐릿 이제 픽사 디즈니 보이 디즈니 픽사 지경 서로 아이덴티티 결과 시너지 실로 다그 사실 인사이드아웃 토피아 증명 토피아 우선 그동안 디즈니 영화 장점 통통 작품 매력 캐릭터 일가견 디즈니 이번 역시 캐릭터 통해 성공 스토리텔링 마치 최대한 사랑 범죄 스릴러 개인 사건 사실 흑막 전개 그동안 만주 토피아 그것 인상 우선 장르 특성 아이 타겟 애니메이션 시도 그동안 무엇 전체 소재 배경 자체 때문 동물 의인화 애니메이션 동물 도로 진화 현재 인간 사회 와동 동물 진보 사회 본능 남아 설정 소재 이야기 개성 또한 영화 요소 고려 캐릭터 행동 자체 디즈니 매력 캐릭터 창조 일가견 때문 캐릭터 가마치 때문 창조 한주 토피아 세계 매력 토피아 디즈니 유의 장점 작품 아이러니 동시 이전 디즈니 영화 다소 다른 양상 주먹 랄프 픽사 스타일 겨울왕국 이전 디즈니 스타일 가지 토피아 모습 모두 스타일 가지 은주 토피아 분위기 세계 주제 식이 태초 동물 약자 사냥 포식자 시간 동물 진화 다른 동물 토피아 포식자 약자 구분 모토 토대 발전 도시 불구 포식자 대한 약자 공포 조사 사건 공포 도시 도시

In [ ]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_review])

# 배열을 출력
print(test_review_tfid)

  (0, 984)	0.025865225062956854
  (0, 981)	0.030426406243689526
  (0, 980)	0.030426406243689526
  (0, 975)	0.030426406243689526
  (0, 972)	0.10346090025182741
  (0, 958)	0.05173045012591371
  (0, 954)	0.030426406243689526
  (0, 948)	0.030426406243689526
  (0, 946)	0.02262901551332726
  (0, 933)	0.030426406243689526
  (0, 923)	0.517248906142722
  (0, 920)	0.030426406243689526
  (0, 916)	0.12932612531478427
  (0, 913)	0.21298484370582668
  (0, 909)	0.025865225062956854
  (0, 906)	0.030426406243689526
  (0, 905)	0.030426406243689526
  (0, 904)	0.030426406243689526
  (0, 899)	0.025865225062956854
  (0, 892)	0.030426406243689526
  (0, 891)	0.030426406243689526
  (0, 889)	0.06085281248737905
  (0, 886)	0.16873885407850814
  (0, 883)	0.06085281248737905
  (0, 875)	0.030426406243689526
  :	:
  (0, 95)	0.05173045012591371
  (0, 88)	0.030426406243689526
  (0, 86)	0.025865225062956854
  (0, 83)	0.025865225062956854
  (0, 81)	0.06085281248737905
  (0, 79)	0.02262901551332726
  (0, 77)	0.0304264062

In [ ]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)[0]
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > '0' else "부정"))

분석 결과 부정적인 리뷰로 예측됩니다. 
